In [218]:
import os
from bs4 import BeautifulSoup

import re

from collections import defaultdict

In [219]:
def extract_numbers_from_html(html_file):
    with open(html_file, 'r', encoding='utf-8') as file:
        soup = BeautifulSoup(file, 'html.parser')
        # print(soup.prettify())

        a_tags = soup.find_all('a')
        file_names = []
        for a_tag in a_tags:
            file_names.append(a_tag.get_text(strip=True).split(':')[0])
        # print(file_names)

        
        # Find all tags with td
        td_tags = soup.find_all('td')
        clone_pairs = 0
        clone_classes = 0
        for td_tag in td_tags:
            if'Clone pairs' in td_tag.get_text(strip=True) :
                clone_pairs = re.search(r'\d+', td_tag.get_text(strip=True)).group()
            if 'Clone classes' in td_tag.get_text(strip=True):
                clone_classes = re.search(r'\d+', td_tag.get_text(strip=True)).group()
        return clone_pairs, clone_classes, '#'.join(file_names)


In [220]:
def traverse_and_extract_numbers(root_folder):
    numbers = []
    for foldername, subfolders, filenames in os.walk(root_folder):
        for filename in filenames:
            if filename.endswith('.html'):
                html_file_path = os.path.join(foldername, filename)
                clone_pairs, clone_classes, file_names = extract_numbers_from_html(html_file_path)
                numbers.append((html_file_path, clone_pairs, clone_classes, file_names))
    return numbers



In [221]:
folders = ['CodeGen_0.6','CodeGen_0.2','StarCoder_0.2','StarCoder_0.8', 'GPT-3.5_1.0']
data = []

for folder in folders:

    root_folder = f'./Output/{folder}/'
    result = traverse_and_extract_numbers(root_folder)
    result.sort(key=lambda x: x[0])

    model = root_folder.split('/')[2].split('_')[0]
    temp = root_folder.split('/')[2].split('_')[1]
    clone_type_counter = defaultdict(list)
    for filename, clone_pairs, clone_classes,file_names in result:
        id = filename.split('/')[3]
        clone_type = filename.split('/')[4].split('.')[0]

        # clone_pairs = int(clone_pairs)
        # clone_classes = int(clone_classes)

        # if clone_type == 'Type-2' and len(clone_type_counter[id]) ==1:
        #     clone_pairs -= clone_type_counter[id][0]
        # if clone_type == 'Type-3' and len(clone_type_counter[id]) ==1:
        #     clone_pairs -= clone_type_counter[id][1]
        # if clone_type == 'Type-3' and len(clone_type_counter[id]) ==2:
        #     clone_pairs -= clone_type_counter[id][0]
        #     clone_pairs -= clone_type_counter[id][1]

        # if clone_pairs == 0:
        #     clone_classes = 0
        
        clone_type_counter[id].append(int(clone_pairs))

        have_clone_original_prompt = False
        have_clone_modified_prompt = False
        
        if '_P' in file_names or '_all' in file_names:
            have_clone_modified_prompt = True

        count_p = file_names.count('_P') + file_names.count('_all')
        if count_p != int(clone_pairs):
            have_clone_original_prompt = True
        data.append((model,temp, id, clone_type, clone_pairs, clone_classes,have_clone_original_prompt,have_clone_modified_prompt, file_names))


In [222]:
import pandas as pd
df = pd.DataFrame(data, columns=['model', 'temp','id', 'clone_type', 'clone_pairs', 'clone_classes','clone_original_prompt','clone_modified_prompt', 'file_names'])
df.to_csv('result.csv', index=False)